In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda")
use_cuda = True
batch_size = 64
test_batch_size = 1000
epochs = 14
log_interval = 100
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
normalized_min = (0 - 0.1307) / 0.3081
normalized_max = (1 - 0.1307) / 0.3081
train_dataset = datasets.MNIST('data', train=True, download=True,
                       transform=transform)
test_dataset = datasets.MNIST('data', train=True, download=True,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, num_workers=2, shuffle=False)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).sum().item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.289829
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.240890
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.496695
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.666997
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.483966
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.228598
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.360383
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.357736
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.349641
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.278837

Test set: Average loss: 0.0002, Accuracy: 56138/60000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.499964
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.150696
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.312374
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.224071
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.290569
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.218040
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.184635
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.140762
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.157989


## Attack

In [16]:
def fgsm_(model, x, target, eps, targeted=True, device='cpu', clip_min=None, clip_max=None):
    """Internal process for all FGSM and PGD attacks."""    
    # create a copy of the input, remove all previous associations to the compute graph...
    input_ = x.clone().detach_().to(device)
    # ... and make sure we are differentiating toward that variable
    input_.requires_grad_()

    # run the model and obtain the loss
    
    model.zero_grad()
    logits = model(input_)
    loss = nn.CrossEntropyLoss()(logits, target)
    loss.backward()
    #perfrom either targeted or untargeted attack
    if targeted:
        out = input_ - eps * input_.grad.sign()
    else:
        out = input_ + eps * input_.grad.sign()
    
    #if desired clip the ouput back to the image domain
    if (clip_min is not None) or (clip_max is not None):
        out.clamp_(min=clip_min, max=clip_max)
    return out


In [17]:
def pgd_(model, x, target, step, eps, iters=7, targeted=True, device='cpu', clip_min=None, clip_max=None):
    projection_min = (x - eps).clamp_(min=clip_min)
    projection_max = (x + eps).clamp_(max=clip_max)
    
    # generate a random point in the +-eps box around x
    offset = torch.rand_like(x)
    offset = (offset*2*eps - eps)
    x = x + offset
    for _ in range(iters):
        x = fgsm_(model, x, target, step, targeted=targeted, device=device, clip_min=None, clip_max=None)
        # project
        x = torch.where(x < projection_min, projection_min, x)
        x = torch.where(x > projection_max, projection_max, x)
        
    if (clip_min is not None) or (clip_max is not None):
        x.clamp_(min=clip_min, max=clip_max)
    return x

In [18]:
def adversarial_accuracy(model, dataset_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(dataset_loader):
        data, target = data.to(device), target.to(device)
        adv = pgd_(model, data, target, 0.1, 0.5, iters=7, targeted=False, device=device, clip_min=normalized_min, clip_max=normalized_max)
        output = model(adv)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        if (batch_idx % 100 == 0):
            print('{} / {}'.format(batch_idx * dataset_loader.batch_size, len(dataset_loader.dataset)))
    print ((correct/len(dataset_loader.dataset) * 100))

In [24]:
adversarial_dataset = torch.utils.data.Subset(test_dataset, [i for i in range(1000)])
adversarial_loader = torch.utils.data.DataLoader(adversarial_dataset, batch_size=2, num_workers=2, shuffle=False)
adversarial_accuracy(model, adversarial_loader)

0 / 1000
200 / 1000
400 / 1000
600 / 1000
800 / 1000
88.7


## Adversarial Training

In [22]:
def adv_train(model, device, train_loader, optimizer, epochs):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs + 1):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            adv_data = pgd_(model, data, target, 0.1, 0.5, iters=7, targeted=False, device=device, clip_min=normalized_min, clip_max=normalized_max)
            optimizer.zero_grad()
            output = model(adv_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

In [23]:
adv_train(model, device, train_loader, optimizer, 2)

Train Epoch: 0 [0/60000 (0%)]	Loss: 1.166683
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.718253
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.672499
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.457467
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.656278
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.622606
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.891890
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.652796
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.674215
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.522886
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.486919
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.349537
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.377484
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.888761
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.392955
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.328932
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.477003
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.580120
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.322453
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.407743
Train Epoch: